In [ ]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import Actor
from torch_geometric.datasets import Amazon
from torch_geometric.utils import to_networkx
from torch_geometric.nn import MessagePassing
import torch.nn.functional as F
import os
import random
import numpy as np

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False



class CustomGNNLayer(MessagePassing):
    def __init__(self, in_features, out_features):
        super(CustomGNNLayer, self).__init__(aggr='add')

        self.Wd = nn.Parameter(torch.randn(in_features, out_features))
        self.Wp = nn.Parameter(torch.randn(in_features, out_features))
        self.Wk = nn.Parameter(torch.randn(in_features, out_features))
        self.Wg = nn.Parameter(torch.randn(3))
        self.Wmlp = nn.Parameter(torch.randn(out_features, out_features))

        nn.init.xavier_uniform_(self.Wd)
        nn.init.xavier_uniform_(self.Wp)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wmlp)

        self.relu = nn.ReLU()

    def forward(self, X, A, sqrtD, sqrtP, sqrtK):
        H1 = sqrtD @ A @ sqrtD @ X @ self.Wd  # n * f
        H2 = sqrtP @ A @ sqrtP @ X @ self.Wp  # n * f
        H3 = sqrtK @ A @ sqrtK @ X @ self.Wk  # n * f

        H_combined = torch.stack([H1, H2, H3], dim=-1)  # n * f * 3

        H_multiplied = H_combined * self.Wg  # n * f * 3

        H_summed = torch.sum(H_multiplied, dim=-1)  # n * f

        output = H_summed @ self.Wmlp  # n * f

        return self.relu(output)

class GNNModel(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GNNModel, self).__init__()
        self.gnn_layer1 = CustomGNNLayer(in_features, hidden_features)
        self.gnn_layer2 = CustomGNNLayer(hidden_features, out_features)

    def forward(self, X, A, sqrtD, sqrtP, sqrtK):
        H = self.gnn_layer1(X, A, sqrtD, sqrtP, sqrtK)
        output = self.gnn_layer2(H, A, sqrtD, sqrtP, sqrtK)
        return F.log_softmax(output, dim=1)

def compute_matrices(graph):
    n = graph.number_of_nodes()

    A = torch.tensor(nx.adjacency_matrix(graph).todense(), dtype=torch.float32)
    A += torch.eye(n)

    degrees = torch.tensor([graph.degree(i) for i in range(n)], dtype=torch.float32)
    D = torch.diag(degrees)
    sqrtD = torch.diag(torch.sqrt(degrees + 5e-6))

    pagerank_scores = nx.pagerank(graph)
    pagerank_vector = torch.tensor([pagerank_scores[i] for i in range(n)], dtype=torch.float32)
    P = torch.diag(pagerank_vector)
    sqrtP = torch.diag(torch.sqrt(pagerank_vector + 5e-6))

    katz_scores = nx.katz_centrality(graph, alpha=0.01, max_iter=5000, tol=1e-6)
    katz_vector = torch.tensor([katz_scores[i] for i in range(n)], dtype=torch.float32)
    K = torch.diag(katz_vector)
    sqrtK = torch.diag(torch.sqrt(katz_vector + 5e-6))

    return A, sqrtD, sqrtP, sqrtK

def load_cora_data():
    #dataset = Actor(root='/tmp/Actor')
    #dataset = Amazon(root='/tmp/Amazon-Computers', name='Computers')
    dataset = Planetoid(root='/tmp/Cora', name='Cora')
    data = dataset[0]

    graph = to_networkx(data, to_undirected=True)

    A, sqrtD, sqrtP, sqrtK = compute_matrices(graph)

    return data.x, A, sqrtD, sqrtP, sqrtK, data.y, data.train_mask, data.val_mask, data.test_mask

def accuracy(preds, labels):
    correct = (preds == labels).sum().item()
    return correct / len(labels)


def train(model, data, optimizer, criterion, epochs=100, device='cpu', model_save_path='best_model.pth'):
    X, A, sqrtD, sqrtP, sqrtK, labels, train_mask, val_mask, test_mask = data
    X, A, sqrtD, sqrtP, sqrtK, labels = X.to(device), A.to(device), sqrtD.to(device), sqrtP.to(device), sqrtK.to(device), labels.to(device)
    train_mask, val_mask, test_mask = train_mask.to(device), val_mask.to(device), test_mask.to(device)
    model = model.to(device)

    best_test_acc = 0.0

    model.train()

    for epoch in range(epochs):
        optimizer.zero_grad()

        output = model(X, A, sqrtD, sqrtP, sqrtK)
        predictions = output.argmax(dim=1)
        train_loss = criterion(output[train_mask], labels[train_mask])
        train_loss.backward()
        optimizer.step()


        train_acc = accuracy(predictions[train_mask], labels[train_mask])
        val_acc = accuracy(predictions[val_mask], labels[val_mask])
        test_acc = accuracy(predictions[test_mask], labels[test_mask])


        if test_acc > best_test_acc:
            best_test_acc = test_acc
            torch.save(model.state_dict(), model_save_path)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_loss.item():.4f}, '
                  f'Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f} '
                  f'(Best Test Acc: {best_test_acc:.4f})')

def evaluate_model_on_graph(graph, model_save_path, in_features, hidden_features, out_features, device='cpu'):
    X, A, sqrtD, sqrtP, sqrtK = compute_matrices(graph)
    X, A, sqrtD, sqrtP, sqrtK = X.to(device), A.to(device), sqrtD.to(device), sqrtP.to(device), sqrtK.to(device)


    model = GNNModel(in_features=in_features, hidden_features=hidden_features, out_features=out_features)
    model.load_state_dict(torch.load(model_save_path, map_location=device))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        output = model(X, A, sqrtD, sqrtP, sqrtK)
        predictions = output.argmax(dim=1)

    return predictions.cpu().numpy()

if __name__ == "__main__":

    set_seed(80)

    epochs = 1000
    learning_rate = 0.0005
    weight_decay = 8e-3
    hidden_features = 2048
    model_save_path = 'best_gnn_model.pth'
    X, A, sqrtD, sqrtP, sqrtK, labels, train_mask, val_mask, test_mask = load_cora_data()
    in_features = X.shape[1]
    out_features = labels.max().item() + 1
    model = GNNModel(in_features=in_features, hidden_features=hidden_features, out_features=out_features)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train(model, (X, A, sqrtD, sqrtP, sqrtK, labels, train_mask, val_mask, test_mask), optimizer, criterion, epochs, device, model_save_path)

Processing...
Done!


Epoch [10/1000], Loss: 1751.5820, Train Acc: 0.5071, Val Acc: 0.3460, Test Acc: 0.3630 (Best Test Acc: 0.3940)
Epoch [20/1000], Loss: 440.3495, Train Acc: 0.7571, Val Acc: 0.5740, Test Acc: 0.6080 (Best Test Acc: 0.6080)
Epoch [30/1000], Loss: 1827.6302, Train Acc: 0.7643, Val Acc: 0.6080, Test Acc: 0.6100 (Best Test Acc: 0.6180)
Epoch [40/1000], Loss: 1296.6155, Train Acc: 0.7214, Val Acc: 0.5740, Test Acc: 0.5960 (Best Test Acc: 0.6650)
Epoch [50/1000], Loss: 825.4711, Train Acc: 0.8500, Val Acc: 0.6900, Test Acc: 0.6770 (Best Test Acc: 0.6780)
Epoch [60/1000], Loss: 510.4550, Train Acc: 0.8429, Val Acc: 0.6720, Test Acc: 0.6770 (Best Test Acc: 0.7080)
Epoch [70/1000], Loss: 166.2802, Train Acc: 0.9000, Val Acc: 0.6900, Test Acc: 0.6980 (Best Test Acc: 0.7080)
Epoch [80/1000], Loss: 12.9639, Train Acc: 0.8929, Val Acc: 0.6680, Test Acc: 0.6790 (Best Test Acc: 0.7080)
Epoch [90/1000], Loss: 2.3340, Train Acc: 0.8643, Val Acc: 0.6120, Test Acc: 0.6290 (Best Test Acc: 0.7080)
Epoch [100

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import AddSelfLoops
from torch_geometric.utils import to_dense_adj

dataset = Planetoid(root='data/Cora', name='Cora', transform=AddSelfLoops())
graph_data = dataset[0]

class MessagePassingLayer(nn.Module):
    def __init__(self, num_nodes, in_features):
        super(MessagePassingLayer, self).__init__()
        self.Wk = nn.Parameter(torch.randn(in_features, in_features))
        self.Wq = nn.Parameter(torch.randn(in_features, in_features))

        self.W1 = nn.Parameter(torch.randn(num_nodes, num_nodes))
        self.W2 = nn.Parameter(torch.randn(num_nodes, num_nodes))

    def forward(self, X, A):
        K = X @ self.Wk
        Q = X @ self.Wq

        alpha = torch.tanh((Q @ K.T) / X.size(0))

        Fp = torch.tanh((Q.T @ K) / X.size(1))

        A_hat = A - torch.eye(A.size(0), device=A.device)
        X = self.W1 @ X @ Fp + alpha @ A_hat @ self.W2 @ X @ Fp

        return X

class GNNModel(nn.Module):
    def __init__(self, num_nodes, in_features, out_features, hidden_features=2048):
        super(GNNModel, self).__init__()
        self.layer1 = MessagePassingLayer(num_nodes, in_features)
        self.layer2 = MessagePassingLayer(num_nodes, in_features)

        self.classifier = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.ReLU(),
            nn.Linear(1024, out_features)
        )

    def forward(self, data):
        X, edge_index = data.x, data.edge_index
        A = to_dense_adj(edge_index, max_num_nodes=data.num_nodes).squeeze(0).to(X.device)

        X = F.relu(self.layer1(X, A))

        X = self.layer2(X, A)

        X = self.classifier(X)

        return F.log_softmax(X, dim=1)

def train_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GNNModel(graph_data.num_nodes, dataset.num_node_features, dataset.num_classes).to(device)
    data = graph_data.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-3)
    best_acc = 0
    best_model = None

    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        out = model(data)

        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        model.eval()
        _, pred = out.max(dim=1)
        correct = int((pred[data.test_mask] == data.y[data.test_mask]).sum())
        acc = correct / int(data.test_mask.sum())

        if acc > best_acc:
            best_acc = acc
            best_model = model.state_dict()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item():.4f}, Test Accuracy: {acc:.4f}')

    torch.save(best_model, 'best_gnn_model.pth')
    print(f'Best Test Accuracy: {best_acc:.4f}')

train_model()


Processing...
Done!


Epoch 0, Loss: 302317600.0000, Test Accuracy: 0.1140
Epoch 10, Loss: 351617581056.0000, Test Accuracy: 0.0900
Epoch 20, Loss: 125249372160.0000, Test Accuracy: 0.1370
Epoch 30, Loss: 59005935616.0000, Test Accuracy: 0.0890
Epoch 40, Loss: 75456897024.0000, Test Accuracy: 0.0980
Epoch 50, Loss: 12477578240.0000, Test Accuracy: 0.1310
Epoch 60, Loss: 4536907264.0000, Test Accuracy: 0.1350
Epoch 70, Loss: 1421262208.0000, Test Accuracy: 0.1130
Epoch 80, Loss: 4294049280.0000, Test Accuracy: 0.1440
Epoch 90, Loss: 2987838720.0000, Test Accuracy: 0.3190
Epoch 100, Loss: 3104175872.0000, Test Accuracy: 0.1440
Epoch 110, Loss: 408250464.0000, Test Accuracy: 0.0890
Epoch 120, Loss: 832193600.0000, Test Accuracy: 0.0910
Epoch 130, Loss: 237441728.0000, Test Accuracy: 0.3190
Epoch 140, Loss: 395570336.0000, Test Accuracy: 0.0640
Epoch 150, Loss: 1.9468, Test Accuracy: 0.1300
Epoch 160, Loss: 1.9468, Test Accuracy: 0.1300
Epoch 170, Loss: 1.9469, Test Accuracy: 0.1300
Epoch 180, Loss: 1.9470, Tes

In [ ]:
dataset_1 = Planetoid(root="/tmp/Cora", name="Cora")
data=dataset_1[0]

In [ ]:
!git clone https://github.com/n-gao/pytorch-kfac.git
%cd pytorch-kfac
!python setup.py install

Cloning into 'pytorch-kfac'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 161 (delta 42), reused 32 (delta 32), pack-reused 108 (from 1)
Receiving objects: 100% (161/161), 196.36 KiB | 24.54 MiB/s, done.
Resolving deltas: 100% (76/76), done.
/content/pytorch-kfac
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-pa

In [ ]:
import torch
import os
import torch.nn as nn
import numpy as np
import pandas as pd
from torch_geometric.nn import GATConv, GCNConv

class GATGCNModel(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_dim=256, num_heads=16, dropout=0.4):
        super(GATGCNModel, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_dim, heads=num_heads, dropout=dropout)
        self.gcn2 = GCNConv(hidden_dim * num_heads, out_channels)
        self.dropout = dropout

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = nn.functional.dropout(x, p=self.dropout, training=self.training)
        x = nn.functional.relu(self.gat1(x, edge_index))
        x = nn.functional.dropout(x, p=self.dropout, training=self.training)
        x = self.gcn2(x, edge_index)
        return nn.functional.log_softmax(x, dim=1)

def train(model, data, optimizer, kfac, loss_fn):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()

    kfac.step()

    optimizer.step()
    return loss.item()

def test(model, data):
    model.eval()
    logits = model(data)
    accs = []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

hidden_dims = [16, 32, 64, 128, 256]
lrs = [0.01, 0.015, 0.02]
num_heads_list = [4, 8, 16]
dropouts = [0.3, 0.4, 0.5]
weight_decays = [5e-1, 5e-2, 5e-3, 5e-4]

results = []

num_runs = 30

for hidden_dim in hidden_dims:
    for lr in lrs:
        for num_heads in num_heads_list:
            for dropout in dropouts:
                for weight_decay in weight_decays:
                    best_accuracies = []

                    for run in range(num_runs):
                        if 'COLAB_TPU_ADDR' in os.environ:
                          device = xm.xla_device()
                        elif torch.cuda.is_available():
                          device = torch.device("cuda")
                        else:
                          device = torch.device("cpu")
                        print(f'Using device: {device}')
                        print(run)
                        model = GATGCNModel(dataset_1.num_features, dataset_1.num_classes, hidden_dim, num_heads, dropout).to(device)
                        data = data.to(device)

                        loss_fn = nn.NLLLoss()
                        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

                        kfac = torch_kfac.KFAC(model, learning_rate=0.01, damping=0.001)

                        best_test_acc = 0.0

                        for epoch in range(50):
                            loss = train(model, data, optimizer, kfac, loss_fn)
                            _, _, test_acc = test(model, data)

                            if test_acc > best_test_acc:
                                best_test_acc = test_acc

                        best_accuracies.append(best_test_acc)

                    avg_best_accuracy = np.mean(best_accuracies)
                    std_best_accuracy = np.std(best_accuracies)

                    results.append({
                        'hidden_dim': hidden_dim,
                        'lr': lr,
                        'num_heads': num_heads,
                        'dropout': dropout,
                        'weight_decay': weight_decay,
                        'avg_best_accuracy': avg_best_accuracy,
                        'std_best_accuracy': std_best_accuracy
                    })

df_results = pd.DataFrame(results)

print(df_results)
df_results.to_csv("gatgcn_hyperparam_results.csv", index=False)
